In [1]:
import sys
#!git clone --recursive https://github.com/dusty-nv/jetson-inference
#!{sys.executable} -m pip install pafy
#!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 101 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
liblapack-dev is already the newest version (3.7.1-4ubuntu1).
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 101 not upgraded.
    100% |################################| 102kB 1.3MB/s a 0:00:01
    100% |################################| 100.2MB 5.2kB/s eta 0:00:01   20% |######                          | 20.7MB 22.5MB/s eta 0:00:04    41% |#############                   | 41.9MB 25.4MB/s eta 0:00:03    45% |##############                  | 45.9MB 24.5MB/s eta 0:00:03    54% |#################               | 54.4MB 24.3MB/s eta 0:00:02    60% |####

In [1]:
import time
from os import getenv
import os
import io
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets as widgets
import traitlets
import PIL
from IPython.display import display, Image
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN
from IPython.display import Image
from my_heatmap_generator import HMap
import cv2

device = torch.device('cuda')
mtcnn = MTCNN(keep_all=True, device=device)
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()

print("model loaded")
outframe = None
url = "https://youtu.be/7PYzSXHd6U4"
load_dotenv()

#defaults
active = "Live"
default_radius_size = getenv('DEFAULT_RADIUS')
default_duration = getenv('DEFAULT_DURATION')
radius = int(getenv('DEFAULT_RADIUS'))
duration = int(getenv('DEFAULT_DURATION'))

def on_value_change(change):
    print("Value change")
    radius = radius_slider.value
    duration = duration_slider.value

duration_slider = widgets.IntSlider(
    value=duration,
    min = 2,
    max = 30,
    step=1,
    description='Airborne Duration:',
    continuous_update=True,
    orientation='horizontal'
)

radius_slider = widgets.IntSlider(
    value=duration,
    min = 10,
    max = 124,
    step=1,
    description='Radius:',
    continuous_update=True,
    orientation='horizontal'
)

model loaded


In [2]:
HMAP = []
camera = USBCamera(width=224, height=224, capture_fps=30, capture_device=0)
camera.running = True
camera.value

def preprocess(image):
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

image_widget = widgets.Image(format='jpeg', width=224, height=224)

def execute(change):
    global hmap, radius, duration
    frame = change['new']
    frame =cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(frame)
    frame = PIL.Image.fromarray(frame)
    if boxes is not None:
        for box in boxes:
            xrange = int((box[2] - box[0])/2)
            yrange = int((box[3] - box[1])/2)
            min_x = box[0] - xrange if box[0]-xrange > 0 else 0
            max_x = box[2] + xrange if box[2]+xrange < 224 else 224
            min_y = box[1] - yrange if box[1]-yrange > 0 else 0
            max_y = box[3] + yrange if box[3]+yrange < 224 else 224
            rectangle = (min_x, min_y, max_x, max_y)
            face_only = frame.crop(rectangle)
            new_width  = int(224 * face_only.size[0] / face_only.size[1])
            face_only = face_only.resize((new_width, 224))
            processed = preprocess(face_only)
            output = model(processed)
            output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
            category_index = output.argmax()
            radius = radius_slider.value
            duration = duration_slider.value
            if output[category_index] < 0.6:
                category_index = 1
            hmap = HMap(box, category_index)
            HMAP.append(hmap)
            frame = hmap.draw_map(frame, radius, duration)
    for hmap in HMAP:
        frame = hmap.draw_map(frame, radius, duration)
    buf = io.BytesIO()
    frame.save(buf, format='JPEG')
    image_widget.value = buf.getvalue()

camera.observe(execute, names='value')
display_handle = display(image_widget, widgets.HBox([duration_slider, radius_slider]))

Image(value=b'', format='jpeg', height='224', width='224')

[0.43975884 0.56024116]
[0.44233486 0.5576651 ]
[0.4319975 0.5680025]
[0.42457542 0.57542455]
[0.38343817 0.6165618 ]
[0.455809   0.54419106]
[0.42601758 0.5739824 ]
[0.4808775 0.5191225]
[0.41258729 0.5874127 ]
[0.37970823 0.62029177]
[0.37424225 0.62575775]
[0.39636794 0.60363203]
[0.38711783 0.6128822 ]
[0.37142548 0.62857455]
[0.3778381 0.6221619]
[0.438594 0.561406]
[0.44850078 0.5514992 ]
[0.39591163 0.60408837]
[0.40054983 0.59945023]
[0.4095811 0.5904189]
[0.40936494 0.59063506]
[0.40282878 0.5971712 ]
[0.41026163 0.58973837]
[0.39984107 0.60015893]
[0.41771498 0.582285  ]
[0.40810117 0.5918988 ]
[0.41427824 0.58572173]
[0.41150528 0.5884948 ]
[0.42160785 0.5783922 ]
[0.40744877 0.59255123]
[0.41852465 0.5814753 ]
[0.41136298 0.58863705]
[0.42090526 0.57909477]
[0.41257188 0.5874281 ]
[0.43206072 0.5679392 ]
[0.47874787 0.52125216]
[0.34352 0.65648]
[0.208719 0.791281]
[0.23497386 0.76502615]
[0.392121   0.60787904]
[0.47485697 0.525143  ]
[0.43860683 0.5613932 ]
[0.45780256 0.